# SPY Spreadsheet

In [1]:
# use future imports for python 3.x forward compatibility
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import

# other imports
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from talib.abstract import *
import itable

# project imports
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [2]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Some global data

In [3]:
symbol = 'SPY'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [4]:
ts = pf.fetch_timeseries(symbol, use_cache=False)

In [5]:
ts.tail()

,high,low,open,close,volume,adj_close
date,,,,,,
2019-12-18,320.25,319.53,320.00,319.59,48133000.00,318.03
2019-12-19,320.98,319.52,319.80,320.90,85310500.00,319.33
2019-12-20,321.97,319.39,320.46,320.73,147142100.00,320.73
2019-12-23,321.65,321.06,321.59,321.22,52990000.00,321.22
2019-12-24,321.52,320.90,321.47,321.24,16464471.00,321.24


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [6]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=True)

In [7]:
ts.head()

,high,low,open,close,volume,adj_close
date,,,,,,
1993-01-29,26.47,26.34,26.47,26.45,1003200.00,26.45
1993-02-01,26.64,26.47,26.47,26.64,480500.00,26.64
1993-02-02,26.72,26.57,26.62,26.70,201300.00,26.70
1993-02-03,27.00,26.72,26.74,26.98,529400.00,26.98
1993-02-04,27.15,26.77,27.07,27.09,531500.00,27.09


Add technical indicator: 200 day MA

In [8]:
sma200 = SMA(ts, timeperiod=200)
ts['sma200'] = sma200

In [9]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200
date,,,,,,,
2019-12-18,318.68,317.97,318.43,318.03,48133000.00,318.03,291.86
2019-12-19,319.41,317.96,318.24,319.33,85310500.00,319.33,292.10
2019-12-20,321.97,319.39,320.46,320.73,147142100.00,320.73,292.36
2019-12-23,321.65,321.06,321.59,321.22,52990000.00,321.22,292.60
2019-12-24,321.52,320.90,321.47,321.24,16464471.00,321.24,292.84


Add technical indicator: ATR

In [10]:
atr = ATR(ts, timeperiod=14)
ts['atr'] = atr

In [11]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr
date,,,,,,,,
2019-12-18,318.68,317.97,318.43,318.03,48133000.00,318.03,291.86,2.10
2019-12-19,319.41,317.96,318.24,319.33,85310500.00,319.33,292.10,2.05
2019-12-20,321.97,319.39,320.46,320.73,147142100.00,320.73,292.36,2.09
2019-12-23,321.65,321.06,321.59,321.22,52990000.00,321.22,292.60,2.01
2019-12-24,321.52,320.90,321.47,321.24,16464471.00,321.24,292.84,1.91


Add technical indicator: 5 day high, and 5 day low

In [12]:
high5 = pd.Series(ts.high).rolling(window=5).max()
low5 = pd.Series(ts.low).rolling(window=5).min()
ts['high5'] = high5
ts['low5'] = low5

In [13]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5
date,,,,,,,,,,
2019-12-18,318.68,317.97,318.43,318.03,48133000.00,318.03,291.86,2.10,318.68,312.63
2019-12-19,319.41,317.96,318.24,319.33,85310500.00,319.33,292.10,2.05,319.41,314.47
2019-12-20,321.97,319.39,320.46,320.73,147142100.00,320.73,292.36,2.09,321.97,315.70
2019-12-23,321.65,321.06,321.59,321.22,52990000.00,321.22,292.60,2.01,321.97,317.92
2019-12-24,321.52,320.90,321.47,321.24,16464471.00,321.24,292.84,1.91,321.97,317.96


Add technical indicator: RSI, and 2-period cumulative RSI

In [14]:
rsi2 = RSI(ts, timeperiod=2)
ts['rsi2'] = rsi2

c2rsi2 = pd.Series(ts.rsi2).rolling(window=2).sum()
ts['c2rsi2'] = c2rsi2

In [15]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2
date,,,,,,,,,,,,
2019-12-18,318.68,317.97,318.43,318.03,48133000.00,318.03,291.86,2.10,318.68,312.63,98.23,196.42
2019-12-19,319.41,317.96,318.24,319.33,85310500.00,319.33,292.10,2.05,319.41,314.47,99.57,197.80
2019-12-20,321.97,319.39,320.46,320.73,147142100.00,320.73,292.36,2.09,321.97,315.70,99.83,199.40
2019-12-23,321.65,321.06,321.59,321.22,52990000.00,321.22,292.60,2.01,321.97,317.92,99.88,199.72
2019-12-24,321.52,320.90,321.47,321.24,16464471.00,321.24,292.84,1.91,321.97,317.96,99.89,199.77


Add technical indicator: Midpoint

In [16]:
mp = (ts.high + ts.low)/2
ts['mp'] = mp

In [17]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp
date,,,,,,,,,,,,,
2019-12-18,318.68,317.97,318.43,318.03,48133000.00,318.03,291.86,2.10,318.68,312.63,98.23,196.42,318.32
2019-12-19,319.41,317.96,318.24,319.33,85310500.00,319.33,292.10,2.05,319.41,314.47,99.57,197.80,318.68
2019-12-20,321.97,319.39,320.46,320.73,147142100.00,320.73,292.36,2.09,321.97,315.70,99.83,199.40,320.68
2019-12-23,321.65,321.06,321.59,321.22,52990000.00,321.22,292.60,2.01,321.97,317.92,99.88,199.72,321.35
2019-12-24,321.52,320.90,321.47,321.24,16464471.00,321.24,292.84,1.91,321.97,317.96,99.89,199.77,321.21


Add technical indicator: SMA10 of midpoint

In [18]:
sma10mp = pd.Series(ts.mp).rolling(window=10).mean()
ts['sma10mp'] = sma10mp

In [19]:
ts.head(10)

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp
date,,,,,,,,,,,,,,
1993-01-29,26.47,26.34,26.47,26.45,1003200.00,26.45,nan,nan,nan,nan,nan,nan,26.41,nan
1993-02-01,26.64,26.47,26.47,26.64,480500.00,26.64,nan,nan,nan,nan,nan,nan,26.56,nan
1993-02-02,26.72,26.57,26.62,26.70,201300.00,26.70,nan,nan,nan,nan,100.00,nan,26.64,nan
1993-02-03,27.00,26.72,26.74,26.98,529400.00,26.98,nan,nan,nan,nan,100.00,200.00,26.86,nan
1993-02-04,27.15,26.77,27.07,27.09,531500.00,27.09,nan,nan,27.15,26.34,100.00,200.00,26.96,nan
1993-02-05,27.13,26.92,27.07,27.07,492100.00,27.07,nan,nan,27.15,26.47,89.34,189.34,27.03,nan
1993-02-08,27.17,27.04,27.07,27.07,596100.00,27.07,nan,nan,27.17,26.57,89.34,178.67,27.10,nan
1993-02-09,26.98,26.83,26.98,26.89,122100.00,26.89,nan,nan,27.17,26.72,16.96,106.30,26.91,nan
1993-02-10,26.94,26.81,26.89,26.92,379600.00,26.92,nan,nan,27.17,26.77,37.28,54.24,26.88,nan


Add technical indicator: Standard Deviation

In [20]:
sd = pd.Series(ts.mp).rolling(window=10).std()
ts['sd'] = sd

In [21]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd
date,,,,,,,,,,,,,,,
2019-12-18,318.68,317.97,318.43,318.03,48133000.00,318.03,291.86,2.10,318.68,312.63,98.23,196.42,318.32,314.48,2.84
2019-12-19,319.41,317.96,318.24,319.33,85310500.00,319.33,292.10,2.05,319.41,314.47,99.57,197.80,318.68,315.36,2.61
2019-12-20,321.97,319.39,320.46,320.73,147142100.00,320.73,292.36,2.09,321.97,315.70,99.83,199.40,320.68,316.11,2.97
2019-12-23,321.65,321.06,321.59,321.22,52990000.00,321.22,292.60,2.01,321.97,317.92,99.88,199.72,321.35,316.95,3.16
2019-12-24,321.52,320.90,321.47,321.24,16464471.00,321.24,292.84,1.91,321.97,317.96,99.89,199.77,321.21,317.86,2.92


In [22]:
upper = ts.sma10mp + ts.sd*2
lower = ts.sma10mp - ts.sd*2
ts['upper'] = upper
ts['lower'] = lower

In [23]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2019-12-18,318.68,317.97,318.43,318.03,48133000.00,318.03,291.86,2.10,318.68,312.63,98.23,196.42,318.32,314.48,2.84,320.16,308.80
2019-12-19,319.41,317.96,318.24,319.33,85310500.00,319.33,292.10,2.05,319.41,314.47,99.57,197.80,318.68,315.36,2.61,320.58,310.13
2019-12-20,321.97,319.39,320.46,320.73,147142100.00,320.73,292.36,2.09,321.97,315.70,99.83,199.40,320.68,316.11,2.97,322.05,310.17
2019-12-23,321.65,321.06,321.59,321.22,52990000.00,321.22,292.60,2.01,321.97,317.92,99.88,199.72,321.35,316.95,3.16,323.27,310.63
2019-12-24,321.52,320.90,321.47,321.24,16464471.00,321.24,292.84,1.91,321.97,317.96,99.89,199.77,321.21,317.86,2.92,323.69,312.02


Select a smaller time from for use with itable

In [24]:
df = ts['2019-01-01':]

In [25]:
df.head()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2019-01-02,246.54,241.38,241.41,245.53,126925200.00,245.53,266.46,6.08,246.73,229.42,84.93,168.27,243.96,242.02,5.16,252.33,231.71
2019-01-03,243.95,239.14,243.62,239.67,144140700.00,239.67,266.35,6.10,246.73,234.52,15.82,100.75,241.55,241.24,4.46,250.15,232.32
2019-01-04,248.41,242.58,242.99,247.70,142628800.00,247.70,266.28,6.29,248.41,239.14,73.94,89.76,245.49,240.97,4.05,249.07,232.87
2019-01-07,251.19,247.01,247.99,249.65,103139100.00,249.65,266.22,6.14,251.19,239.14,80.49,154.43,249.10,241.66,4.80,251.27,232.06
2019-01-08,252.53,249.28,252.05,252.00,102512600.00,252.00,266.20,5.93,252.53,239.14,87.84,168.33,250.90,242.73,5.58,253.88,231.57


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [26]:
pt = itable.PrettyTable(df, tstyle=itable.TableStyle(theme='theme1'), center=True, header_row=True, rpt_header=20)

pt.update_col_header_style(format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] == 'volume':
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high5'][row] == pt.df['high'][row]) and \
       (pt.df['high5'][row] > pt.df['high'][row-1]):
        col = df.columns.get_loc('high5')    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low5'][row] == pt.df['low'][row]) and \
       (pt.df['low5'][row] < pt.df['low'][row-1]):
        col = df.columns.get_loc('low5')
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')          

In [27]:
pt

,HIGH,LOW,OPEN,CLOSE,VOLUME,ADJ_CLOSE,SMA200,ATR,HIGH5,LOW5,RSI2,C2RSI2,MP,SMA10MP,SD,UPPER,LOWER
2019/01/02,246.54,241.38,241.41,245.53,126925200,245.53,266.46,6.08,246.73,229.42,84.93,168.27,243.96,242.02,5.16,252.33,231.71
2019/01/03,243.95,239.14,243.62,239.67,144140700,239.67,266.35,6.10,246.73,234.52,15.82,100.75,241.55,241.24,4.46,250.15,232.32
2019/01/04,248.41,242.58,242.99,247.70,142628800,247.70,266.28,6.29,248.41,239.14,73.94,89.76,245.49,240.97,4.05,249.07,232.87
2019/01/07,251.19,247.01,247.99,249.65,103139100,249.65,266.22,6.14,251.19,239.14,80.49,154.43,249.10,241.66,4.80,251.27,232.06
2019/01/08,252.53,249.28,252.05,252.00,102512600,252.00,266.20,5.93,252.53,239.14,87.84,168.33,250.90,242.73,5.58,253.88,231.57
2019/01/09,254.10,251.43,252.77,253.18,95006600,253.18,266.22,5.70,254.10,239.14,91.17,179.01,252.76,244.69,5.27,255.23,234.14
2019/01/10,254.34,250.75,251.50,254.07,96823900,254.07,266.21,5.55,254.34,242.58,93.77,184.94,252.55,246.39,4.70,255.79,236.99
2019/01/11,254.20,252.25,252.89,254.17,73858100,254.17,266.22,5.29,254.34,247.01,94.15,187.91,253.22,247.80,4.38,256.56,239.05
2019/01/14,253.50,251.64,252.09,252.62,70908200,252.62,266.23,5.09,254.34,249.28,32.27,126.42,252.57,248.63,4.42,257.47,239.79
2019/01/15,255.86,253.02,253.03,255.51,85208300,255.51,266.23,4.96,255.86,250.75,80.39,112.67,254.44,249.66,4.47,258.59,240.72
